## DO NOT RUN THE CELLS. CODE is included here for you to refer to for your future projects.

# Max Voting 

from sklearn.ensemble import VotingClassifier

model1 = LogisticRegression(random_state=1)

model2 = tree.DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

model.fit(x_train,y_train)

model.score(x_test,y_test)

# Averaging

model1 = tree.DecisionTreeClassifier()

model2 = KNeighborsClassifier()

model3= LogisticRegression()

model1.fit(x_train,y_train)

model2.fit(x_train,y_train)

model3.fit(x_train,y_train)

pred1=model1.predict_proba(x_test)

pred2=model2.predict_proba(x_test)

pred3=model3.predict_proba(x_test)

finalpred=(pred1+pred2+pred3)/3

# Weighted Average

model1 = tree.DecisionTreeClassifier()

model2 = KNeighborsClassifier()

model3= LogisticRegression()

model1.fit(x_train,y_train)

model2.fit(x_train,y_train)

model3.fit(x_train,y_train)

pred1=model1.predict_proba(x_test)

pred2=model2.predict_proba(x_test)

pred3=model3.predict_proba(x_test)

finalpred=(pred1*0.3+pred2*0.3+pred3*0.4)

In [65]:
# Stacking 
# Stacking method given model, train and y (series) data set, test set, number of splits
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pandas as pd
def Stacking(model,train,y,test,n_fold):
    # generates folds as StratifiedKFold which can be used to split data into equal subsets, if possible.
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    #creates empty numpy vector for test predicitions
    test_pred=np.empty((test.shape[0],1),float)
    #creates empty numpy vector for train predicitions
    train_pred=np.empty((0,1),float)
    #generates splits of train and y by shuffling index-sets
    for train_indices,val_indices in folds.split(train,y.values):
        # fixes x_train and x_val givfen previous indices to generatre another subset
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        # fixes y_train and y_val givfen previous indices to generatre another subset
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]
        # the given model is fitted with x and y train
        model.fit(X=x_train,y=y_train)
        # prediction array gains element x_val
        train_pred=np.append(train_pred,model.predict(x_val))
        # predection given the given test set
        test_pred=np.append(test_pred,model.predict(test))
    # return test and train predicitons
    #return test_pred.reshape(-1,1),train_pred
    return test_pred,train_pred

In [67]:
# Creating 2 base models
# Decision tree as model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
model1 = DecisionTreeClassifier(random_state=1)
# X set
X = pd.Series([x for x in range(80)])
import random
y = pd.Series([round(random.uniform(0, 1)) for _ in X])
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 5)
#X_train, X_test, y_train, y_test 
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
x_test = pd.DataFrame(x_test)
y_test= pd.DataFrame(y_test)
#print(len(x_test), len(x_train))
#generating test and train set for 10 folds given the DecisionTree model
test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=x_train,test=x_test,y=y_train)
print(len(x_test), len(x_train), len(test_pred1), len(train_pred1))
#casting sets to DataFrame
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)
# KNN as secondary model
model2 = KNeighborsClassifier()
# Stacking for k nearest neighbors
test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)
# again casting
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)


# KNN reinitialization #double?
#model2 = KNeighborsClassifier()
# another KNN test
#test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)

#train_pred2=pd.DataFrame(train_pred2)
#test_pred2=pd.DataFrame(test_pred2)

20 60 220 60


/Users/peterniksa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/peterniksa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/peterniksa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/peterniksa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/peterniksa/anaconda3/lib/python3.7/site-p

In [69]:
#Create a third model, logistic regression, on the predictions of the decision tree and knn models.
# create dataframe with pred1 and pred2 for train and test
from sklearn.linear_model import LogisticRegression
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)
# initialize Logistic Regression model
model = LogisticRegression(random_state=1)
# fit logitstic regression using the stacked x data sets for y_train, generate the score
model.fit(df,y_train)
# evaluate model based on df and y with y_test and df_test
print(len(df_test), len(y_test), len(test_pred1), len(train_pred1))
df_test
model.score(df_test[:len(y_test)], y_test)

220 20 220 60


/Users/peterniksa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/peterniksa/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6

In order to simplify the above explanation, the stacking model we have created has only two levels. The decision tree and knn models are built at level zero, while a logistic regression model is built at level one. Feel free to create multiple levels in a stacking model.

 

One may generate additionally model runs and stack them and even use the first stacked as an intermediary step which is used in a final regression.